In [35]:
import boto3
import sagemaker
import pandas as pd
from sagemaker import image_uris
from sagemaker  import TrainingInput
from sagemaker.session import Session
from sklearn.datasets import load_iris

In [36]:
X, y = load_iris(return_X_y=True, as_frame=True)
dados = pd.concat([y, X], axis=1)
dados.to_csv("data/dados.csv", header=False, index=False)

In [37]:
bucket = sagemaker.Session().default_bucket()

In [38]:
xgboost_container = sagemaker.image_uris.retrieve("xgboost", "us-east-1", "1.2-1")

In [39]:
role = "arn:aws:iam::885248014373:role/service-role/AmazonSageMaker-ExecutionRole-20210305T230941"

In [40]:
# initialize hyperparameters
hyperparameters = {
    "objective":"multi:softmax",
    "num_round":"2",
    "num_class": 3
}

In [41]:
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          role=role,
                                          hyperparameters=hyperparameters,
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5,
                                          output_path=f"s3://{bucket}")

In [42]:
input_data = sagemaker.Session().upload_data(path="data", bucket=bucket)
input_data

's3://sagemaker-us-east-1-885248014373/data'

In [43]:
train_input = TrainingInput(input_data, content_type="csv")

In [44]:
estimator.fit({'train': train_input})

2021-03-09 14:43:40 Starting - Starting the training job...
2021-03-09 14:44:03 Starting - Launching requested ML instancesProfilerReport-1615301019: InProgress
......
2021-03-09 14:45:13 Starting - Preparing the instances for training...
2021-03-09 14:45:52 Downloading - Downloading input data...
2021-03-09 14:46:25 Training - Downloading the training image..[2021-03-09 14:46:34.831 ip-10-0-106-166.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:S

UnexpectedStatusException: Error for Training job sagemaker-xgboost-2021-03-09-14-43-39-381: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_xgboost_container/algorithm_mode/train.py", line 226, in train_job
    verbose_eval=False)
  File "/miniconda3/lib/python3.7/site-packages/xgboost/training.py", line 212, in train
    xgb_model=xgb_model, callbacks=callbacks)
  File "/miniconda3/lib/python3.7/site-packages/xgboost/training.py", line 75, in _train_internal
    bst.update(dtrain, i, obj)
  File "/miniconda3/lib/python3.7/site-packages/xgboost/core.py", line 1161, in update
    dtrain.handle))
  File "/miniconda3/lib/python3.7/site-packages/xgboost/core.py", line 188, in _check_call
    raise XGBoostError(py_str(_LIB.XGBGetLastError()))
xgboost.core.XGBoostError: [14:46:34] ../src/objective/multiclass_obj.cu:117: SoftmaxMultiClassObj: label must be in [0, num_class).
Stack trace:
  [bt] (0)

In [ ]:
estimator.deploy(initial_instance_count=1,
                 instance_type="ml.t2.medium", 
                 data_capture_config = data_capture_config)

In [ ]:
endpointName = "sagemaker-xgboost-2021-03-09-03-10-34-846"

In [ ]:
features = pd.DataFrame(X)
features.to_csv("data/features.csv", header=False, index=False)

In [ ]:
with open("data/features.csv") as f:
    er = f.read()

In [ ]:
import boto3

client = boto3.client("sagemaker-runtime")

for line in  er.splitlines():
    response = client.invoke_endpoint(EndpointName=endpointName,
                                      Body=line,
                                     ContentType="csv")

In [ ]:
response["Body"].read().decode("utf-8")